In [1]:
%load_ext autoreload
%autoreload 2

In [125]:
import json 
import numpy as np 
from arcs.setup_functions import ReactionGibbsandEquilibrium
from monty.serialization import loadfn

In [126]:
dft_data = loadfn('../tests/test_dft_data.json')
rge = ReactionGibbsandEquilibrium(dft_data)
gibbs_free_energy = rge.Gibbs(dft_dict=dft_data['CO2'],temperature=100,pressure=1)
gibbs_free_energy


-31.974842281859573

In [127]:
from reactit import ReactionGenerator

rg = ReactionGenerator(compounds=['CO','CO2','H2O','H2'])
rg.iterate()
reactions = rg.as_dict()
reaction_gibbs = rge.reaction_gibbs(reaction=reactions[0],temperature=100,pressure=1)
reaction_gibbs

100%|██████████| 2/2 [00:00<00:00, 538.18it/s]


-0.6184489112909475

In [128]:
dft_data['reactions']

{'0': {'reaction_string': '2 H2S + 3 O2 = 2 H2SO3',
  'reactants': {'H2S': 2, 'O2': 3},
  'products': {'H2SO3': 2}},
 '1': {'reaction_string': '2 H2O = 1 O2 + 2 H2',
  'reactants': {'H2O': 2},
  'products': {'O2': 1, 'H2': 2}},
 '2': {'reaction_string': '1 CO2 + 1 CH4 = 1 CH3COOH',
  'reactants': {'CO2': 1, 'CH4': 1},
  'products': {'CH3COOH': 1}},
 '3': {'reaction_string': '1 CO + 1 CH3OH = 2 H2CO',
  'reactants': {'CO': 1, 'CH3OH': 1},
  'products': {'H2CO': 2}},
 '4': {'reaction_string': '1 O2 + 2 HNO2 = 2 HNO3',
  'reactants': {'O2': 1, 'HNO2': 2},
  'products': {'HNO3': 2}},
 '5': {'reaction_string': '1 O2 + 1 CH3COOH = 2 CH2O2',
  'reactants': {'O2': 1, 'CH3COOH': 1},
  'products': {'CH2O2': 2}},
 '6': {'reaction_string': '8 O2 + 1 S8 = 8 SO2',
  'reactants': {'O2': 8, 'S8': 1},
  'products': {'SO2': 8}},
 '7': {'reaction_string': '1 H2S + 2 O2 = 1 H2SO4',
  'reactants': {'H2S': 1, 'O2': 2},
  'products': {'H2SO4': 1}},
 '8': {'reaction_string': '2 CH3CHO + 1 O2 = 2 CH3COOH',
  '

In [ ]:
import tqdm 
results = [rge.get_reaction_gibbs_and_equilibrium(reaction=r,temperature=100,pressure=1) for r in tqdm.tqdm(dft_data['reactions'].values())]

100%|██████████| 501/501 [00:00<00:00, 2565.64it/s]


In [124]:

def testfunc(reaction):
    if isinstance(reaction,list):
        print('is list')
    elif isinstance(reaction,Equilibrium):
        print('is chempy.Equilibrium')
    elif isinstance(reaction,str):
        print('is string')

testfunc('CO + H2O = H2 + CO2')

is string
